In [3]:
"""Importing libraries"""
import time
import os
from collections import Counter
import numpy as np
import cv2
import rasterio # pylint: disable=import-error
import tifffile # pylint: disable=import-error
from sklearn.cluster import KMeans # pylint: disable=import-error
from sklearn.decomposition import PCA # pylint: disable=import-error
from osgeo import gdal
def convert_gray_to_rgb(image):
    """Conversion of single band image to three band image"""
    if len(image.shape) == 2:
        return np.broadcast_to(image[:, :, np.newaxis], (*image.shape, 3))
    return image
def calculate_index(band1, band2, formula):
    """Generalizing index calculation function"""
    b_1 = band1.astype('float64')
    b_2 = band2.astype('float64')
    return formula(b_1,b_2)
def process_image(bands):
    """NDVI, NDWI and NDSI calculation"""
    ndvi = calculate_index(bands['nir'], bands['red'],
    lambda nir, red: np.where((nir + red) == 0., 0, (nir - red) / (nir + red)))
    ndwi = calculate_index(bands['green'], bands['nir'],
    lambda green, nir: np.where((green + nir) == 0., 0, (green - nir) / (green + nir)))
    ndsi = calculate_index(bands['swir1'], bands['nir'],
    lambda swir1, nir: np.where((swir1 + nir) == 0., 0, (swir1 - nir) / (swir1 + nir)))
    return ndvi, ndwi, ndsi
def create_masks(ndvi, ndwi, ndsi):
    """Creating masks based on thresholds for both images"""
    vegetation_mask = ndvi > 0.090
    water_mask = ndwi > 0.125
    shadow_mask = ndsi > 0.5
    combined_mask = np.logical_or(np.logical_or(vegetation_mask, water_mask), shadow_mask)
    return combined_mask
def apply_mask(image, mask):
    """Applying combined mask to each image"""
    mask = mask[:, :, np.newaxis]
    return np.where(mask, np.nan, image.astype('float64'))
def load_optical_tif_images(image_path1, image_path2, apply_preprocessing=False):
    """Load optical TIFF and TIF images"""
    def read_image(image_path):
        with rasterio.open(image_path) as src:
            bands = src.read()
        return bands.transpose(1, 2, 0)
    image1 = read_image(image_path1)
    image2 = read_image(image_path2)
    if image1 is None or image2 is None:
        print(f"Error: Unable to load images {image_path1} or {image_path2}")
        return None, None
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0])) # pylint: disable=no-member
    if not apply_preprocessing:
        return image1, image2
    bands1 = {
        'red': image1[:, :, 0], 'green': image1[:, :, 1], 'blue': image1[:, :, 2],
        'nir': image1[:, :, 3], 'swir1': image1[:, :, 4], 'swir2': image1[:, :, 5]
    }
    bands2 = {
        'red': image2[:, :, 0], 'green': image2[:, :, 1], 'blue': image2[:, :, 2],
        'nir': image2[:, :, 3], 'swir1': image2[:, :, 4], 'swir2': image2[:, :, 5]
    }
    ndvi1, ndwi1, ndsi1 = process_image(bands1)
    ndvi2, ndwi2, ndsi2 = process_image(bands2)
    mask1 = create_masks(ndvi1, ndwi1, ndsi1)
    mask2 = create_masks(ndvi2, ndwi2, ndsi2)
    masked_image1 = apply_mask(image1, mask1)
    masked_image2 = apply_mask(image2, mask2)
    masked_image1 = np.nan_to_num(masked_image1, nan=0, posinf=0, neginf=0)
    masked_image2 = np.nan_to_num(masked_image2, nan=0, posinf=0, neginf=0)
    return masked_image1, masked_image2
def load_sar_tif_images(image_path1, image_path2):
    """Load SAR TIFF and TIF images"""
    image1 = tifffile.imread(image_path1)
    image2 = tifffile.imread(image_path2)
    if image1 is None or image2 is None:
        print(f"Error: Unable to load images {image_path1} or {image_path2}")
        return None, None
    image1 = convert_gray_to_rgb(image1)
    image2 = convert_gray_to_rgb(image2)
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0])) # pylint: disable=no-member
    return image1, image2
def load_optical_ntf_images(image_path1, image_path2, apply_preprocessing=False):
    """Load optical NITF and NTF images"""
    dataset_1 = gdal.Open(image_path1)
    dataset_2 = gdal.Open(image_path2)
    image1 = dataset_1.ReadAsArray().transpose(1, 2, 0)
    image2 = dataset_2.ReadAsArray().transpose(1, 2, 0)
    if image1 is None or image2 is None:
        print(f"Error: Unable to load images {image_path1} or {image_path2}")
        return None, None
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0])) # pylint: disable=no-member
    if not apply_preprocessing:
        return image1, image2
    bands1 = {
        'red': image1[:, :, 0], 'green': image1[:, :, 1], 'blue': image1[:, :, 2],
        'nir': image1[:, :, 3], 'swir1': image1[:, :, 4], 'swir2': image1[:, :, 5]
    }
    bands2 = {
        'red': image2[:, :, 0], 'green': image2[:, :, 1], 'blue': image2[:, :, 2],
        'nir': image2[:, :, 3], 'swir1': image2[:, :, 4], 'swir2': image2[:, :, 5]
    }
    ndvi1, ndwi1, ndsi1 = process_image(bands1)
    ndvi2, ndwi2, ndsi2 = process_image(bands2)
    mask1 = create_masks(ndvi1, ndwi1, ndsi1)
    mask2 = create_masks(ndvi2, ndwi2, ndsi2)
    masked_image1 = apply_mask(image1, mask1)
    masked_image2 = apply_mask(image2, mask2)
    masked_image1 = np.nan_to_num(masked_image1, nan=0, posinf=0, neginf=0)
    masked_image2 = np.nan_to_num(masked_image2, nan=0, posinf=0, neginf=0)
    return masked_image1, masked_image2
def load_sar_ntf_images(image_path1, image_path2):
    """Load SAR NITF and NTF images"""
    dataset_1 = gdal.Open(image_path1)
    dataset_2 = gdal.Open(image_path2)
    image1 = dataset_1.ReadAsArray()
    image2 = dataset_2.ReadAsArray()
    if image1 is None or image2 is None:
        print(f"Error: Unable to load images {image_path1} or {image_path2}")
        return None, None
    image1 = convert_gray_to_rgb(image1)
    image2 = convert_gray_to_rgb(image2)
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0])) # pylint: disable=no-member
    return image1, image2
def load_png_jpg_bmp_images(image_path1, image_path2):
    """Load PNG, JPG, JPEG, and BMP images"""
    image1 = cv2.imread(image_path1) # pylint: disable=no-member
    image2 = cv2.imread(image_path2) # pylint: disable=no-member
    if image1 is None or image2 is None:
        print(f"Error: Unable to load images {image_path1} or {image_path2}")
        return None, None
    image1 = convert_gray_to_rgb(image1)
    image2 = convert_gray_to_rgb(image2)
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0])) # pylint: disable=no-member
    return image1, image2
def process_tif(input_path):
    """Process TIFF and TIF image and return it as an RGB array"""
    with rasterio.open(input_path) as src:
        num_bands = src.count
        is_grayscale = num_bands < 3
        if is_grayscale:
            red = np.nan_to_num(src.read(1), nan=0)
            green = blue = red
        else:
            red = np.nan_to_num(src.read(1), nan=0)
            green = np.nan_to_num(src.read(2), nan=0)
            blue = np.nan_to_num(src.read(3), nan=0)
        min_val = min(red.min(), green.min(), blue.min())
        max_val = max(red.max(), green.max(), blue.max())
        red = ((red - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        green = ((green - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        blue = ((blue - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        rgb = np.dstack((red, green, blue))
    return rgb, is_grayscale
def process_ntf(input_path):
    """Process NITF and NTF image and return it as an RGB array"""
    dataset = gdal.Open(input_path)
    num_bands = dataset.RasterCount
    is_grayscale = num_bands < 3
    if is_grayscale:
        red = dataset.GetRasterBand(1).ReadAsArray()
        green = blue = red
    else:
        red = dataset.GetRasterBand(1).ReadAsArray()
        green = dataset.GetRasterBand(2).ReadAsArray()
        blue = dataset.GetRasterBand(3).ReadAsArray()
    red = np.nan_to_num(red, nan=0)
    green = np.nan_to_num(green, nan=0)
    blue = np.nan_to_num(blue, nan=0)
    min_val = min(red.min(), green.min(), blue.min())
    max_val = max(red.max(), green.max(), blue.max())
    if max_val > min_val:
        red = ((red - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        green = ((green - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        blue = ((blue - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    else:
        red = green = blue = np.zeros_like(red, dtype=np.uint8)
    rgb = np.dstack((red, green, blue))
    return rgb, is_grayscale
def automatic_brightness_contrast(image, clip_hist_percent=2.5):
    """Automatically adjusts brightness and contrast to preserve color properties"""
    if len(image.shape) == 3:
        zero_pixels = np.count_nonzero(np.all(image == 0, axis=2))
        total_pixels = image.shape[0] * image.shape[1]
    else:
        zero_pixels = np.count_nonzero(image == 0)
        total_pixels = image.size
    zero_percentage = zero_pixels / total_pixels * 100
    if zero_percentage < 10:
        clip_hist_percent = 2.0
        contrast_boost = 1.0
        brightness_adjustment = 0.0
        strategy_name = "Low Zeros"
    elif 10 <= zero_percentage < 30:
        clip_hist_percent = 2.3
        contrast_boost = 1.0
        brightness_adjustment = 0.0
        strategy_name = "Medium Zeros"
    elif 30 <= zero_percentage < 50:
        clip_hist_percent = 1.8
        contrast_boost = 1.0
        brightness_adjustment = 0.0
        strategy_name = "High Zeros"
    else:
        clip_hist_percent = 1.8
        contrast_boost = 1.0
        brightness_adjustment = 0.0
        strategy_name = "Extreme Zeros"
    print(f"Selected Strategy: {strategy_name}")
    if len(image.shape) == 3 and image.shape[2] > 3:
        process_image = image[:, :, :3].copy()
    else:
        process_image = image.copy()
    process_image = process_image.astype(np.float32)
    global_alpha, global_beta = 1.0, 0.0
    channel_count = min(3, process_image.shape[2]) if len(process_image.shape) == 3 else 1
    for i in range(channel_count):
        channel = process_image[:, :, i] if len(process_image.shape) == 3 else process_image
        non_zero_mask = channel > 0
        hist, _ = np.histogram(channel[non_zero_mask].flatten(), bins=256, range=[0, 256])
        cdf = hist.cumsum()
        total_non_zero_pixels = channel[non_zero_mask].size
        clip_value = clip_hist_percent * total_non_zero_pixels / 100.0
        minimum = np.argmax(cdf > clip_value)
        maximum = np.argmax(cdf > cdf[-1] - clip_value)
        if maximum > minimum:
            alpha = (255.0 / (maximum - minimum)) * contrast_boost
            beta = -minimum * alpha + brightness_adjustment
            channel_adjusted = channel * alpha + beta
            channel[non_zero_mask] = np.clip(channel_adjusted[non_zero_mask], 0, 255)
            global_alpha *= alpha
            global_beta += beta
            darken_threshold = 50
            lighten_threshold = 200
            channel[non_zero_mask & (channel < darken_threshold)] *= 0.9
            mask = non_zero_mask & (channel > lighten_threshold)
            channel[mask] += (255 - channel[mask]) * 0.2
            zero_pixel_factor = 0.7 + (0.1 * (brightness_adjustment / 20))
            channel[~non_zero_mask] *= zero_pixel_factor
            if len(process_image.shape) == 3:
                process_image[:, :, i] = np.clip(channel, 0, 255)
            else:
                process_image = np.clip(channel, 0, 255)
    enhanced_image = np.clip(process_image, 0, 255).astype(np.uint8)
    return enhanced_image, global_alpha, global_beta
def find_vector_set(diff_image, nw_size):
    """Finding vector set"""
    i = 0
    j = 0
    vector_set = np.zeros((int(nw_size[0] * nw_size[1] / 25), 25))
    while i < vector_set.shape[0]:
        while j < nw_size[1]:
            k = 0
            while k < nw_size[0]:
                block = diff_image[j:j + 5, k:k + 5]
                feature = block.ravel()
                vector_set[i, :] = feature
                k = k + 5
            j = j + 5
        i = i + 1
    mean_vec = np.mean(vector_set, axis=0)
    vector_set = vector_set - mean_vec
    return vector_set, mean_vec
def find_fvs(evs, diff_image, mean_vec, new):
    """Finding feature vector set"""
    i = 2
    feature_vector_set = []
    while i < new[1] - 2:
        j = 2
        while j < new[0] - 2:
            block = diff_image[i - 2:i + 3, j - 2:j + 3]
            feature = block.flatten()
            feature_vector_set.append(feature)
            j = j + 1
        i = i + 1
    fvs = np.dot(feature_vector_set, evs)
    fvs = fvs - mean_vec
    return fvs
def clustering(fvs, components, new):
    """K-means clustering"""
    kmeans = KMeans(components, verbose=0)
    kmeans.fit(fvs)
    output = kmeans.predict(fvs)
    count = Counter(output)
    small_index = min(count, key=count.get)
    change_mask = np.reshape(output, (new[1] - 4, new[0] - 4))
    return small_index, change_mask
def overlay_tif_image(change_map_resized, image1, rgb_image_tif):
    """Overlay for TIFF and TIF images"""
    _, binary_mask = cv2.threshold(change_map_resized, 200, 255, cv2.THRESH_BINARY) # pylint: disable=no-member
    green_mask = np.zeros_like(rgb_image_tif)
    indices = np.where(binary_mask == 255)
    green_mask[indices[0], indices[1], :] = [0, 255, 0]
    overlay = cv2.add(rgb_image_tif, green_mask) # pylint: disable=no-member
    return overlay
def overlay_ntf_image(change_map_resized, image1, rgb_image_ntf):
    """Overlay for NITF and NTF images"""
    _, binary_mask = cv2.threshold(change_map_resized, 200, 255, cv2.THRESH_BINARY) # pylint: disable=no-member
    green_mask = np.zeros_like(rgb_image_ntf)
    indices = np.where(binary_mask == 255)
    green_mask[indices[0], indices[1], :] = [0, 255, 0]
    overlay = cv2.add(rgb_image_ntf, green_mask) # pylint: disable=no-member
    return overlay
def overlay_png_jpg_bmp_image(change_map_resized, image1):
    """Overlay for PNG, JPG, JPEG, and BMP images"""
    _, binary_mask = cv2.threshold(change_map_resized, 200, 255, cv2.THRESH_BINARY) # pylint: disable=no-member
    green_mask = np.zeros_like(image1)
    indices = np.where(binary_mask == 255)
    green_mask[indices[0], indices[1], :] = [0, 255, 0]
    overlay = cv2.add(image1, green_mask) # pylint: disable=no-member
    return overlay
def save_nitf_ntf_image(output_path, overlay, ref_image_path):
    """Save overlay as a NITF and NTF image"""
    dataset = gdal.Open(ref_image_path)
    driver = gdal.GetDriverByName("NITF")
    out_dataset = driver.Create(output_path, dataset.RasterXSize,
                                dataset.RasterYSize, 3, gdal.GDT_Byte)
    for i in range(3):
        out_dataset.GetRasterBand(i+1).WriteArray(overlay[:, :, i])
    out_dataset.FlushCache()
def run_change_detection(image_path1, image_path2):
    """Main execution block"""
    start_time = time.time()
    file_ext1 = os.path.splitext(image_path1)[-1].lower()
    file_ext2 = os.path.splitext(image_path2)[-1].lower()
    if file_ext1 != file_ext2:
        print("Error: Image formats do not match")
        return
    if file_ext1 in ['.jpg', '.jpeg', '.bmp', '.png']:
        image1, image2 = load_png_jpg_bmp_images(image_path1, image_path2)
    elif file_ext1 in ['.tif', '.tiff']:
        with rasterio.open(image_path1) as src1:
            bands1 = src1.read()
        with rasterio.open(image_path2) as src2:
            bands2 = src2.read()
        num_bands1 = bands1.shape[0]
        num_bands2 = bands2.shape[0]
        if num_bands1 == 1 and num_bands2 == 1:
            print("Detected: single-band image")
            image1, image2 = load_sar_tif_images(image_path1, image_path2)
        else:
            print("Detected: multi-band image")
            user_input = int(input("1. Without preprocessing, 2. With preprocessing: "))
            if user_input == 1:
                image1, image2 = load_optical_tif_images(image_path1,
                                                         image_path2, apply_preprocessing=False)
            else:
                image1, image2 = load_optical_tif_images(image_path1,
                                                         image_path2, apply_preprocessing=True)
    elif file_ext1 in ['.ntf', '.nitf']:
        dataset_1 = gdal.Open(image_path1)
        dataset_2 = gdal.Open(image_path2)
        num_bands1 = dataset_1.RasterCount
        num_bands2 = dataset_2.RasterCount
        if num_bands1 == 1 and num_bands2 == 1:
            print("Detected: single-band image")
            image1, image2 = load_sar_ntf_images(image_path1, image_path2)
        else:
            print("Detected: multi-band image")
            user_input = int(input("1. Without preprocessing, 2. With preprocessing: "))
            if user_input == 1:
                image1, image2 = load_optical_ntf_images(image_path1,
                                                         image_path2, apply_preprocessing=False)
            else:
                image1, image2 = load_optical_ntf_images(image_path1,
                                                         image_path2, apply_preprocessing=True)
    new_size = np.asarray(image1.shape) // 5
    new_size = new_size.astype(int) *5
    image1_resized = cv2.resize(image1, (new_size[0],new_size[1])).astype(int) # pylint: disable=no-member
    image2_resized = cv2.resize(image2, (new_size[0],new_size[1])).astype(int) # pylint: disable=no-member
    difference_image = abs(image1_resized - image2_resized)
    difference_image = difference_image[:, :, 1]
    vector_sets, mean_vector = find_vector_set(difference_image, new_size)
    pca = PCA()
    pca.fit(vector_sets)
    eigen_vector = pca.components_
    feature_vector = find_fvs(eigen_vector, difference_image, mean_vector, new_size)
    components = 3
    least_index, change_map = clustering(feature_vector, components, new_size)
    change_map[change_map == least_index] = 255
    change_map[change_map != 255] = 0
    change_map = change_map.astype(np.uint8)
    original_height, original_width = image1.shape[:2]
    change_map_resized = cv2.resize(change_map, (original_width, original_height)) # pylint: disable=no-member
    end_time = time.time()
    print("Computational time in seconds:", end_time - start_time)
    output_path = None
    if image_path1.lower().endswith(('.tif', '.tiff')):
        rgb_image_tif, is_grayscale = process_tif(image_path1)
        if not is_grayscale:
            rgb_image_tif, _, _ = automatic_brightness_contrast(rgb_image_tif)
        overlay = overlay_tif_image(change_map_resized, image1, rgb_image_tif)
        output_path = 'Output_image.tif'
        tifffile.imwrite(output_path, overlay)
    elif image_path1.lower().endswith(('.nitf')):
        rgb_image_ntf, is_grayscale = process_ntf(image_path1)
        if not is_grayscale:
            rgb_image_ntf, _, _ = automatic_brightness_contrast(rgb_image_ntf)
        overlay = overlay_ntf_image(change_map_resized, image1, rgb_image_ntf)
        output_path = 'Output_image.nitf'
        save_nitf_ntf_image(output_path, overlay, image_path1)
    elif image_path1.lower().endswith(('.ntf')):
        rgb_image_ntf, is_grayscale = process_ntf(image_path1)
        if not is_grayscale:
            rgb_image_ntf, _, _ = automatic_brightness_contrast(rgb_image_ntf)
        overlay = overlay_ntf_image(change_map_resized, image1, rgb_image_ntf)
        output_path = 'Output_image.ntf'
        save_nitf_ntf_image(output_path, overlay, image_path1)
    else:
        overlay = overlay_png_jpg_bmp_image(change_map_resized, image1)
        output_path = 'Output_image.png'
        cv2.imwrite(output_path, overlay) # pylint: disable=no-member
    print(f'Output saved to {output_path}')
image_path1 = 'Pisa_1.tif'  # Path of Image_1
image_path2 = 'Pisa_2.tif'  # Path of Image_2
run_change_detection(image_path1, image_path2)


Detected: multi-band image
1. Without preprocessing, 2. With preprocessing: 2
Computational time in seconds: 6.171787261962891
Selected Strategy: Low Zeros
Output saved to Output_image.tif
